# IMDB Random Forests Regression
![ImdbIcon](../images/imdbheader.jpg)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report

C:\Users\nolan_fur2pfn\.conda\envs\dsi\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\nolan_fur2pfn\.conda\envs\dsi\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\nolan_fur2pfn\.conda\envs\dsi\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
revenue_df = pd.read_csv('../data/revenue_df.csv')

In [3]:
revenue_df.columns

Index(['movie_title', 'year', 'tagline', 'plot', 'cast', 'duration', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Drama',
       'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western', 'avg_vote',
       'total_votes', 'us_voters_votes', 'votes', 'votes_1', 'votes_2',
       'votes_3', 'votes_4', 'votes_5', 'votes_6', 'votes_7', 'votes_8',
       'votes_9', 'votes_10', 'popularity', 'director_score', 'actor_score',
       'actress_score', 'tagline_sentiment', 'plot_sentiment', 'total_score',
       'profitable', 'budget', 'revenue', 'budget_adj', 'revenue_adj'],
      dtype='object')

## Random Forests Regression

In [246]:
features = ['duration', 'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 
            'History', 'Horror', 'Music', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'avg_vote', 'total_votes', 'votes', 
            'votes_1', 'votes_2', 'votes_5', 'votes_6',  'votes_8', 'votes_9', 'popularity',  'director_score', 
            'actor_score', 'actress_score', 'plot_sentiment']

In [247]:
X = revenue_df[features]
y = revenue_df['revenue']

In [248]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = 42)

In [249]:
rf = RandomForestRegressor()

In [250]:
rf.fit(X_train, y_train)

RandomForestRegressor()

In [251]:
print("Training Score:", rf.score(X_train, y_train))
print(" ")
print("Testing Score:", rf.score(X_test, y_test))

Training Score: 0.9500578936307246
 
Testing Score: 0.5868542676010903


In [252]:
print("Cross Val Score:", cross_val_score(rf, X_train, y_train, cv = 5).mean())

Cross Val Score: 0.6264799223010684


In [253]:
model_params = {}
count = 0

In [261]:
rf_params = {
    'max_depth': [8, 10, 12],
    'min_samples_leaf': [0, 1, 2],
    'min_samples_split': [4, 6, 8],
    'n_estimators': [300, 350, 400, 450],
}

gs = GridSearchCV(rf, param_grid=rf_params, cv = 5, n_jobs = 12)

gs.fit(X_train, y_train)

count += 1

gs.best_params_['best_score'] = gs.best_score_

gs.best_params_['training_score'] = gs.score(X_train, y_train)

gs.best_params_['testing_score'] = gs.score(X_test, y_test)

model_params[f'model_{count}'] = gs.best_params_

model_df = pd.DataFrame.from_dict(model_params, orient='index')

model_df

,max_depth,min_samples_leaf,min_samples_split,n_estimators,best_score,training_score,testing_score
model_1,2,1,2,300,0.498220,0.614353,0.477783
model_2,6,1,4,300,0.627643,0.866092,0.575893
model_3,8,1,4,400,0.631705,0.902036,0.578664
model_4,8,1,4,400,0.635804,0.903137,0.582779
model_5,8,1,6,400,0.632236,0.890721,0.581981
model_6,8,1,8,350,0.632934,0.883201,0.576246
model_7,10,1,6,400,0.632982,0.909729,0.581727
model_8,10,1,4,450,0.636133,0.921790,0.585710


## AdaBoostRegressor

In [18]:
model_params_two = {}
count_two = 0

In [22]:
ada = AdaBoostRegressor(base_estimator=RandomForestRegressor())

ada_params = {
    'base_estimator__max_depth': [3, 4, 5],
    'learning_rate': [.80, .85, .90],
    'n_estimators': [300, 400, 500],
}

gs_two = GridSearchCV(ada, param_grid=ada_params, cv=3, n_jobs = 12)

gs_two.fit(X_train, y_train)

count_two += 1

gs_two.best_params_['best_score'] = gs_two.best_score_

gs_two.best_params_['training_score'] = gs_two.score(X_train, y_train)

gs_two.best_params_['testing_score'] = gs_two.score(X_test, y_test)

model_params_two[f'model_{count_two}'] = gs_two.best_params_

model_df_two = pd.DataFrame.from_dict(model_params_two, orient='index')

model_df_two

,base_estimator__max_depth,learning_rate,n_estimators,best_score,training_score,testing_score
model_1,3,1.00,300,0.600364,0.768746,0.427752
model_2,4,0.95,300,0.619965,0.833536,0.490926
model_3,4,0.90,400,0.640080,0.842121,0.455173
model_4,4,0.80,500,0.644468,0.851919,0.468607
